# Testing graph and agent architecture

In [1]:
from langgraph.graph import StateGraph, END
from typing import Annotated, Literal, TypedDict
from pydantic import BaseModel, Field
import json
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, ToolMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_linkup import LinkupSearchTool
from langgraph.checkpoint.memory import InMemorySaver

from IPython.display import Image, Markdown, display
from uuid import uuid4

GEMINI_MODEL_NAME = "gemini-2.5-flash-lite-preview-06-17"
#GEMINI_MODEL_NAME = "gemini-2.5-flash"

In [2]:
model = ChatGoogleGenerativeAI(model=GEMINI_MODEL_NAME, temperature=0.0, thinking_budget=0)

## Prompts

In [ ]:
# Project-specific constants
ENGINE_OR_FRAMEWORK = "Love2D framework"
PROG_LANGUAGE = "Lua"

TECH_ARCHITECT_WRITER_SYSTEM_PROMPT = (
    "You are an expert technical architect in game development using the " + ENGINE_OR_FRAMEWORK
    + " with " + PROG_LANGUAGE + " as the programming language. Given the feature request below, "
    "create a high-level technical architecture following this exact structure:\n"
    "## System Overview\n"
    "Write 2-3 sentences summarizing what this system does and its main purpose.\n"
    "\n"
    "## Core Components\n"
    "For each major component, include:\n"
    " - Component name\n"
    " - Purpose: [1 sentence describing what it does]\n"
    " - Key Methods: [list 3-5 main functions with brief descriptions]\n"
    " - Data Structures: [main tables/variables it manages]\n"
    "\n"
    "## Component Interactions\n"
    "Describe how components communicate with each other. Include:\n"
    "- Which component calls which\n"
    "- What data is passed between them\n"
    "- Event flow from user input to system response\n"
    "\n"
    "## File Organization\n"
    "List the specific files to create and what each contains.\n"
    "\n"
    "## Integration Points\n"
    "Specify how this connects to existing game systems (player stats, save system, input handling, etc.).\n"
    "\n"
    "Requirements:\n"
    "- Include all functionality mentioned in the feature request\n"
    "- Each component should have a single, clear responsibility\n"
    "- Use standard " + ENGINE_OR_FRAMEWORK + " patterns, files, and configurations\n"
    "- Specify data structures as " + PROG_LANGUAGE + " tables with key field names\n"
    "- Keep components loosely coupled with clear interfaces\n"
    "\n"
    "Do not include implementation details, specific code, or functionality not requested.\n"
    "-----\n"
)

TECH_ARCHITECT_REVIEWER_SYSTEM_PROMPT = (
    "You are an expert technical architect in game development using the " + ENGINE_OR_FRAMEWORK
    + " with " + PROG_LANGUAGE + " as the programming language. Review the technical architecture "
    "against the original feature request. Check each item below:\n"
    "\n"
    "**Completeness Check:**\n"
    "1. Are all features from the request addressed in the architecture?\n"
    "2. Does each component have a clear purpose and key methods listed?\n"
    "3. Is component interaction described (who calls what)?\n"
    "4. Are file names and organization specified?\n"
    "5. Are integration points with existing systems identified?\n"
    "\n"
    "**Feasibility Check:**\n"
    "1. Are the proposed components appropriate for the " + ENGINE_OR_FRAMEWORK + " and the " + PROG_LANGUAGE +" language?\n"
    "2. Are the data structures feasible (using " + PROG_LANGUAGE + " formats properly)?\n"
    "3. Are the component interactions realistic and efficient?\n"
    "\n"
    "**Quality Check:**\n"
    "1. Is each component focused on a single responsibility?\n"
    "2. Are components loosely coupled?\n"
    "3. Is the architecture as simple as possible while meeting requirements?\n"
    "\n"
    "Your response must comply with this output structure:\n"
    #" - 'review_status': 'Approved' or 'Needs Revision',\n"
    " - 'completeness_score': [1-5, where 5 = all features covered],\n"
    " - 'feasibility_score': [1-5, where 5 = fully feasible in " + ENGINE_OR_FRAMEWORK + "/" + PROG_LANGUAGE + "],\n"
    " - 'simplicity_score': [1-5, where 5 = appropriately simple],\n"
    " - 'review_notes': Clearly describe what changes are required to improve this FIP\n"
    #" - 'review_notes': If 'Needs Revision': specific issues to fix. If 'Approved': leave empty string.\n"
    "\n"
    "Example output format:\n"
    #"review_status: 'Approved'\n"
    "completeness_score: 5\n"
    "feasibility_score: 4\n"
    "simplicity_score: 5\n"
    "review_notes: ''\n"
    "\n"
    #"Only approve if ALL scores are 4 or greater. Otherwise mark as 'Needs Revision' and provide specific feedback.\n"
    "-----\n"
)

FIP_WRITER_SYSTEM_PROMPT = (
    "You are an expert game developer using the " + ENGINE_OR_FRAMEWORK + " with "
    + PROG_LANGUAGE + " as the programming language. Given the technical architecture below, "
    "create a feature implementation plan (FIP) following this exact structure:\n"
    "\n"
    "## Implementation Overview\n"
    "Write 2-3 sentences describing what will be implemented and the overall approach.\n"
    "\n"
    "## Implementation Phases\n"
    "Break work into phases. For each phase:\n"
    "\n"
    "**Phase [X]: [Name] (Days X-Y)**\n"
    " - **Priority:** Critical/High/Medium/Low\n"
    " - **Files to create:** [list specific " + PROG_LANGUAGE + " filenames]\n"
    " - **Files to modify:** [list existing files and what changes]\n"
    " - **Key functions to implement:** [function names with brief descriptions]\n"
    " - **Dependencies:** [what must be completed first]\n"
    "\n"
    "## Integration Instructions\n"
    " - **Where to call new code:** [specific files and functions]\n"
    " - **Required imports:** [require statements needed]\n"
    " - **Initialization:** [where to create instances]\n"
    " - **Game loop integration:** [update/draw call locations]\n"
    "\n"
    "## Testing Requirements\n"
    "For each major component:\n"
    " - **Unit tests:** [specific functions to test]\n"
    " - **Integration tests:** [end-to-end workflows to verify]\n"
    " - **Manual testing steps:** [how to verify functionality works]\n"
    "\n"
    "## File Structure\n"
    "```\n"
    "[project_folder]/\n"
    "├── [component1]\n"
    "├── [component2]\n"
    "└── data/\n"
    "    ├── [datafiles]\n"
    "```\n"
    "\n"
    "Requirements:\n"
    " - Include implementation steps for ALL components from the architecture\n"
    " - Specify exact function names and file locations\n"
    " - Provide concrete testing steps that can be executed\n"
    " - Include all integration points with existing systems\n"
    " - Order phases by dependencies (prerequisites first)\n"
    " - Each phase should take 1-3 days maximum\n"
    "\n"
    "Do not include actual code implementation - focus on what to build and how to integrate it.\n"
    "-----\n"
)

FIP_REVIEWER_SYSTEM_PROMPT = (
    "You are an expert game developer using the " + ENGINE_OR_FRAMEWORK + " with "
    + PROG_LANGUAGE + " as the programming language. Review the feature implementation "
    "plan (FIP) against the technical architecture. Check each item:\n"
    "\n"
    "**Completeness Check:**\n"
    " 1. Does each architecture component have implementation steps in the FIP?\n"
    " 2. Are all files to create/modify specified with names?\n"
    " 3. Are function names and integration points clearly stated?\n"
    " 4. Are testing requirements specified for each component?\n"
    " 5. Is the file structure/organization defined?\n"
    "\n"
    "**Implementation Quality Check:**\n"
    " 1. Are phases ordered by dependencies (no circular dependencies)?\n"
    " 2. Are phase durations realistic (1-3 days each)?\n"
    " 3. Are integration instructions specific (exact files/functions)?\n"
    " 4. Are testing steps actionable and concrete?\n"
    "\n"
    "**Architecture Alignment Check:**\n"
    " 1. Does the FIP address all components from the architecture?\n"
    " 2. Are the proposed functions aligned with architecture methods?\n"
    " 3. Are data structures and interactions preserved?\n"
    "\n"
    "Your response must comply with this output structure:\n"
    #" - 'review_status': 'Approved' or 'Needs Revision',\n"
    " - 'completeness_score': [1-5, where 5 = all architecture components covered],\n"
    " - 'implementation_score': [1-5, where 5 = clear, actionable instructions],\n"
    " - 'alignment_score': [1-5, where 5 = fully matches architecture],\n"
    " - 'review_notes': Clearly describe what changes are required to improve this FIP\n"
    #" - 'review_notes': If 'Needs Revision': specific missing items or issues. If 'Approved': leave empty string.\n"
    "\n"
    "Example output format:\n"
    #"review_status: 'Approved'\n"
    "completeness_score: 5\n"
    "implementation_score: 4\n"
    "alignment_score: 5\n"
    "review_notes: ''\n"
    "\n"
    #"Only approve if ALL scores are 4 or greater. Otherwise mark as 'Needs Revision' and provide specific feedback.\n"
    "-----\n"
)


## State Classes

In [4]:
class ArchitectureReviewOutput(BaseModel):
    review_status: Literal["Approved", "Needs Revision"] = Field(description="Status of the review, either 'Approved' or 'Needs Revision'.", default="Needs Revision")
    review_notes: str = Field(description="Detailed feedback on the document being reviewed. If the document is approved, this should be an empty string.", default="")
    completeness_score: int = Field(ge=1, le=5, description="Score for completeness", default=1)
    feasibility_score: int = Field(ge=1, le=5, description="Score for feasibility", default=1)
    simplicity_score: int = Field(ge=1, le=5, description="Score for simplicity", default=1)

class FIPReviewOutput(BaseModel):
    review_status: Literal["Approved", "Needs Revision"] = Field(description="Status of the review, either 'Approved' or 'Needs Revision'.", default="Needs Revision")
    review_notes: str = Field(description="Detailed feedback on the document being reviewed. If the document is approved, this should be an empty string.", default="")
    completeness_score: int = Field(ge=1, le=5, description="Score for completeness", default=1)
    implementation_score: int = Field(ge=1, le=5, description="Score for implementation quality", default=1)
    alignment_score: int = Field(ge=1, le=5, description="Score for architecture alignment", default=1)

class Review(BaseModel):
    """Review ouptut format for the review agents."""
    document_type: Literal["architecture_plan", "feature_implementation_plan"]
    review_output: ArchitectureReviewOutput | FIPReviewOutput
    review_count: int = 0
    max_reviews_allowed: int = 3

class WorkflowState(BaseModel):
    """
    Represents the state of the total workflow. This is the highest level state.
    """
    feature_request: str= ""
    architecture_plan: str = ""
    feature_implementation_plan: str = ""
    ap_review: Review = Field(default_factory=lambda: Review(
        document_type="architecture_plan",
        review_output=ArchitectureReviewOutput(
            review_status="Needs Revision",
            review_notes="",
            completeness_score=1,
            feasibility_score=1,
            simplicity_score=1
        )
    ))
    fip_review: Review = Field(default_factory=lambda: Review(
        document_type="feature_implementation_plan",
        review_output=FIPReviewOutput(
            review_status="Needs Revision",
            review_notes="",
            completeness_score=1,
            implementation_score=1,
            alignment_score=1
        )
    ))

## Nodes

In [5]:
def parse_review_response(content: str, review_type: str) -> ArchitectureReviewOutput | FIPReviewOutput:
    lines = content.strip().split('\n')
    data = {}
    
    for line in lines:
        if ':' in line:
            key, value = line.split(':', 1)
            key = key.strip().lower().replace(' ', '_')
            value = value.strip()
            
            if key == 'review_status':
                data[key] = value
            elif key.endswith('_score'):
                data[key] = int(value) if value.isdigit() else 1
            elif key == 'review_notes':
                data[key] = value
    
    if review_type == "architecture":
        return ArchitectureReviewOutput(**data)
    else:
        return FIPReviewOutput(**data)

def tech_arch_writer_node(state: WorkflowState) -> dict:
    """
    Node to write the technical architecture.
    """
    # Create the system message with the prompt
    system_msg = SystemMessage(content=TECH_ARCHITECT_WRITER_SYSTEM_PROMPT)
    feature_content = f"Feature request:\n{state.feature_request}"
    
    # If you need tools, bind them to the model
    #tools = [LinkupSearchTool()]  # Add your tools here
    model_with_tools = model #.bind_tools(tools)
    
    # Create messages list
    if state.ap_review.review_count > 0:
        previous_draft = f"\nThis is the previous draft of the architecture plan:\n{state.architecture_plan}\n\n"
        review_feedback = f"Review feedback to address:\n{state.ap_review.review_output.review_notes}\n\n"
        feature_msg = HumanMessage(content=feature_content + previous_draft + review_feedback)
    else:
        feature_msg = HumanMessage(content=feature_content)
    
    messages = [system_msg, feature_msg]
    
    # Invoke the model
    response = model_with_tools.invoke(messages)

    if hasattr(response, 'usage_metadata') and response.usage_metadata:
        print(f"Tech Arch Writer - Input tokens: {response.usage_metadata.get('input_tokens', 'N/A')}")
        print(f"Tech Arch Writer - Output tokens: {response.usage_metadata.get('output_tokens', 'N/A')}")
        print(f"Tech Arch Writer - Total tokens: {response.usage_metadata.get('total_tokens', 'N/A')}")
    
    # Return the update to state
    return {"architecture_plan": response.content}

def tech_arch_validator_node(state: WorkflowState) -> dict:
    """
    Node to validate the technical architecture.
    """
    print(f"Starting architecture validation...")
    # Create the system message
    system_msg = SystemMessage(content=TECH_ARCHITECT_REVIEWER_SYSTEM_PROMPT)
    feature_req = f"Original feature request:\n{state.feature_request}\n-----\n"
    ap_draft = f"Technical architecture to review:\n{state.architecture_plan}\n-----\n"
    task_msg = HumanMessage(content=feature_req + ap_draft)
    
    # Use structured output to ensure proper formatting
    model_with_structured_output = model #.with_structured_output(ArchitectureReviewOutput)
    
    # Add tools if needed (bind tools before structured output)
    #tools = [LinkupSearchTool()]
    model_with_tools_and_structure = model_with_structured_output#.bind_tools(tools)
    
    # Invoke model
    response = model_with_tools_and_structure.invoke([system_msg, task_msg])
    parsed_response = parse_review_response(response.content, "architecture")
    print(parsed_response)
    #d = eval(response.content.split("json")[-1].split("```")[0])
    #review_output = ArchitectureReviewOutput(
    #    review_status=d.get("review_status", "Needs Revision"),
    #    review_notes=d.get("review_notes", ""),
    #    completeness_score=d.get("completeness_score", 1),
    #    feasibility_score=d.get("feasibility_score", 1),
    #    simplicity_score=d.get("simplicity_score", 1)
    #)

    if hasattr(response, 'usage_metadata') and response.usage_metadata:
        print(f"Tech Arch Reviewer - Input tokens: {response.usage_metadata.get('input_tokens', 'N/A')}")
        print(f"Tech Arch Reviewer - Output tokens: {response.usage_metadata.get('output_tokens', 'N/A')}")
        print(f"Tech Arch Reviewer - Total tokens: {response.usage_metadata.get('total_tokens', 'N/A')}")
    # response is now a ReviewOutput object, no parsing needed!
    updated_review = Review(
        document_type="architecture_plan",
        review_output=parsed_response,
        review_count=state.ap_review.review_count + 1,
        max_reviews_allowed=state.ap_review.max_reviews_allowed
    )
    if parsed_response.review_status == "Needs Revision":
        print(f"Architecture review notes: {parsed_response.review_notes}")
    
    return {"ap_review": updated_review}

def feature_impl_writer(state: WorkflowState) -> dict:
    """
    Node to write the feature implementation plan. Includes the subgraph to both write the plan and
    call tools.
    """
    system_msg = SystemMessage(content=FIP_WRITER_SYSTEM_PROMPT)
    task_content = f"Technical architecture:\n{state.architecture_plan}"
    #tools = [LinkupSearchTool()]  # Add your tools here
    model_with_tools = model#.bind_tools(tools)

    if state.fip_review.review_count > 0:
        previous_draft = f"Previously submitted draft of the feature implementation plan:\n{state.feature_implementation_plan}\n\n"
        review_feedback = f"Please address this feedback in your next draft of the feature implementation plan:\n{state.fip_review.review_output.review_notes}\n\n"
        task_msg = HumanMessage(content=task_content + previous_draft + review_feedback)
    else:
        task_msg = HumanMessage(content=task_content)
    
    messages = [system_msg, task_msg]
    # Invoke the model
    response = model_with_tools.invoke(messages)
    return {"feature_implementation_plan": response.content}

def feature_impl_validator_node(state: WorkflowState) -> dict:
    """
    Node to validate the feature implementation plan. Includes the subgraph to both review the plan and
    call tools.
    """
    system_msg = SystemMessage(content=FIP_REVIEWER_SYSTEM_PROMPT)
    ap_draft = f"Technical architecture:\n{state.architecture_plan}\n-----\n"
    fip_draft = f"Feature implementation plan to review:\n{state.feature_implementation_plan}\n-----\n"
    task_msg = HumanMessage(content=ap_draft + fip_draft)
    model_with_structured_output = model #.with_structured_output(FIPReviewOutput)
    #tools = [LinkupSearchTool()]  # Add your tools here
    model_with_tools_and_structure = model_with_structured_output#.bind_tools(tools)
    response = model_with_tools_and_structure.invoke([system_msg, task_msg])
    parsed_response = parse_review_response(response.content, "fip")
    #d = eval(response.content.split("json")[-1].split("```")[0])
    #print(d)
    #review_output = FIPReviewOutput(
    #    review_status=d.get("review_status", "Needs Revision"),
    #    review_notes=d.get("review_notes", ""),
    #    completeness_score=d.get("completeness_score", 1),
    #    implementation_score=d.get("implementation_score", 1),
    #    alignment_score=d.get("alignment_score", 1)
    #)
    updated_review = Review(
        document_type="feature_implementation_plan",
        review_output=parsed_response,
        review_count=state.fip_review.review_count + 1,
        max_reviews_allowed=state.fip_review.max_reviews_allowed
    )
    if parsed_response.review_status == "Needs Revision":
        print(f"Feature implementation plan review notes: {parsed_response.review_notes}")
    return {"fip_review": updated_review}

def tech_arch_approval(state: WorkflowState) -> str:
    """
    Node to approve the technical architecture.
    """
    if state.ap_review.review_count >= state.ap_review.max_reviews_allowed:
        return "Review limit reached"
    else:
        return state.ap_review.review_output.review_status

def feature_impl_approval(state: WorkflowState) -> str:
    """
    Node to approve the feature implementation plan.
    """
    if state.fip_review.review_count >= state.fip_review.max_reviews_allowed:
        return "Review limit reached"
    else:
        return state.fip_review.review_output.review_status

## Graphs

In [6]:
# Create the workflow graph
workflow_builder = StateGraph(WorkflowState)

# Add nodes
workflow_builder.add_node("tech_arch_writer", tech_arch_writer_node)
workflow_builder.add_node("tech_arch_validator", tech_arch_validator_node)
workflow_builder.add_node("feature_impl_writer", feature_impl_writer)
workflow_builder.add_node("feature_impl_validator", feature_impl_validator_node)

# Add edges
workflow_builder.add_edge("tech_arch_writer", "tech_arch_validator")
workflow_builder.add_edge("feature_impl_writer", "feature_impl_validator")

# Add control flow
workflow_builder.set_entry_point("tech_arch_writer")
workflow_builder.add_conditional_edges(
    "tech_arch_validator",
    tech_arch_approval,
    {
        "Approved": "feature_impl_writer",
        "Needs Revision": "tech_arch_writer",
        "Review limit reached": END
    }
)
workflow_builder.add_conditional_edges(
    "feature_impl_validator",
    feature_impl_approval,
    {
        "Approved": END,
        "Needs Revision": "feature_impl_writer",
        "Review limit reached": END
    }
)

# Compile the graph
workflow_graph = workflow_builder.compile(
    name="Feature Development Workflow"
)

In [7]:
#response = workflow_graph.invoke({"feature_request": "Implement a dash feature for the first person movement controller."})
response = workflow_graph.invoke({"feature_request": """Overview
Implement a comprehensive player inventory system that allows players to collect, manage, and use items throughout the game. This system should support both consumable and equipment items with a user-friendly interface.
Functional Requirements
Core Inventory Management

Grid-based storage: 20-slot inventory grid (4x5 layout) where each item occupies one slot
Item stacking: Consumable items stack up to 99 per slot, equipment items don't stack
Item sorting: Automatic sorting option that groups items by type (consumables, weapons, armor, misc)
Item filtering: Search functionality to quickly locate specific items by name or type

Item Types and Properties

Consumable Items: Health potions, mana potions, food items with immediate use effects
Equipment Items: Weapons, armor pieces, accessories that modify player stats when equipped
Quest Items: Special items that cannot be dropped or sold, marked with unique visual indicator
Crafting Materials: Basic materials used in future crafting system (prepare data structure)

User Interface

Inventory Window: Accessible via 'I' key, displays item grid with item icons and quantity
Item Tooltips: Hover over items to show name, description, stats, and value
Context Menu: Right-click items for options (use, drop, examine)
Drag & Drop: Click and drag items to rearrange inventory slots
Equipment Slots: Separate UI panel showing currently equipped weapon and armor

Item Acquisition & Removal

Item Pickup: Players walk over dropped items to automatically collect them
Item Dropping: Drop items from inventory to the game world at player location
Item Usage: Consumables apply effects immediately and are removed from inventory
Equipment: Click equipment items to equip/unequip, replacing current equipment

Technical Requirements
Data Persistence

Save/load inventory state between game sessions
Maintain item properties and quantities accurately
Handle equipped items persistence

Performance Constraints

Inventory operations should not cause frame drops
Support up to 1000 different item types in game data
Efficient item lookup and manipulation algorithms

Integration Points

Player Stats System: Equipment items modify player health, damage, speed stats
Game World: Items can be dropped and picked up from the environment
Future Systems: Data structure should accommodate future crafting and trading systems

User Experience Goals

Intuitive drag-and-drop interface similar to popular RPG games
Clear visual feedback for all inventory operations
Responsive controls with no input lag
Consistent visual design matching game's art style

Success Criteria

Players can successfully manage 20+ items without confusion
All inventory operations complete within 50ms
No item duplication or loss bugs
Save/load maintains exact inventory state
UI remains usable at all supported screen resolutions (800x600 to 1920x1080)"""})

Tech Arch Writer - Input tokens: 851
Tech Arch Writer - Output tokens: 2059
Tech Arch Writer - Total tokens: 2910
Starting architecture validation...
review_status='Needs Revision' review_notes='' completeness_score=1 feasibility_score=1 simplicity_score=1
Tech Arch Reviewer - Input tokens: 2975
Tech Arch Reviewer - Output tokens: 216
Tech Arch Reviewer - Total tokens: 3191
Architecture review notes: 
Tech Arch Writer - Input tokens: 2929
Tech Arch Writer - Output tokens: 2389
Tech Arch Writer - Total tokens: 5318
Starting architecture validation...
review_status='Needs Revision' review_notes='' completeness_score=1 feasibility_score=1 simplicity_score=1
Tech Arch Reviewer - Input tokens: 3305
Tech Arch Reviewer - Output tokens: 202
Tech Arch Reviewer - Total tokens: 3507
Architecture review notes: 
Tech Arch Writer - Input tokens: 3259
Tech Arch Writer - Output tokens: 2399
Tech Arch Writer - Total tokens: 5658
Starting architecture validation...
review_status='Needs Revision' review_

In [8]:
for k,v in response.items():
    print(f"\n---\n{k}")
    print(v)


---
feature_request
Overview
Implement a comprehensive player inventory system that allows players to collect, manage, and use items throughout the game. This system should support both consumable and equipment items with a user-friendly interface.
Functional Requirements
Core Inventory Management

Grid-based storage: 20-slot inventory grid (4x5 layout) where each item occupies one slot
Item stacking: Consumable items stack up to 99 per slot, equipment items don't stack
Item sorting: Automatic sorting option that groups items by type (consumables, weapons, armor, misc)
Item filtering: Search functionality to quickly locate specific items by name or type

Item Types and Properties

Consumable Items: Health potions, mana potions, food items with immediate use effects
Equipment Items: Weapons, armor pieces, accessories that modify player stats when equipped
Quest Items: Special items that cannot be dropped or sold, marked with unique visual indicator
Crafting Materials: Basic materials u

In [9]:
r = """```json
{
  "review_status": "Approved",
  "completeness_score": 5,
  "feasibility_score": 5,
  "simplicity_score": 4,
  "review_notes": ""
}
```"""

In [10]:
d = eval(r.split("json")[-1].split("```")[0])  # Extract JSON content from the string